In [1]:
from core import database
import isb
import pandas as pd
import h3
import plotly.express as px
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()

In [ ]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Task exception was never retrieved
future: <Task finished name='Task-104' coro=<RefreshAheadCache._schedule_refresh.<locals>._refresh_task() done, defined at C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py:142> exception=RefreshError('Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.')>
Traceback (most recent call last):
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 152, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 107, in _perform_refresh
    connection_info = await self._client.get_connection_info(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\co

In [ ]:
df = db.query("""
select 
  r.realestate_id, 
  l.listing_id,
  l.span as listing_span, 
  lp.span as listing_price_span, 
  lp.price, 
  u.district_id,
  r.square_meters, 
  r.construction_year,
  rt.name as realestate_type_name, 
  c.name as city_name,
  a.name as address_name,
  a.street_name,
  case when construction_year >= extract(year from now()) - 5 and (sa.first_sale_date is null or sa.first_sale_date >= now()) then 'new' else 'old' end as construction_category,
  price, r.square_meters,
  (now()::date - lower(l.span)::date)::int as days_listed,
  ST_X(u.origin) as latitude, 
  ST_Y(u.origin) as longitude
from listings l
join listing_prices lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join realestates r
on lp.realestate_id = r.realestate_id
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
  and now()::date <@ lp.span
""")

['makro-website:europe-west2:makro-prod']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 152, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 107, in _perform_refresh
    connection_info = await self._client.get_connection_info(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\client.py", line 274, in get_connection_info
    self._credentials.refresh(requests.Request())
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\oauth2\credentials.py", line 409, in refresh
    ) = reauth.refresh_grant(


In [4]:
df\
.groupby(['street_name', 'city_name', 'construction_category'])\
.agg(
    days_listed = ('days_listed', 'mean'),
    total = ('realestate_id', 'nunique')
)\
.sort_values("days_listed")\
.loc[lambda r: r.total > 1]\
.tail(40)

days_listed  \
street_name      city_name               construction_category                
Fálkahlíð        Reykjavíkurborg         new                     287.181818   
Hávegur          Kópavogsbær             new                     290.727273   
Kleppsmýrarvegur Reykjavíkurborg         new                     292.000000   
Dugguvogur       Reykjavíkurborg         new                     294.071429   
Álftröð          Kópavogsbær             new                     295.156250   
Arkarvogur       Reykjavíkurborg         new                     296.545455   
Apalskarð        Hafnarfjarðarkaupstaður old                     299.250000   
                                         new                     305.173913   
Bleikjukvísl     Reykjavíkurborg         old                     307.000000   
Dunhagi          Reykjavíkurborg         old                     310.500000   
Tryggvagata      Reykjavíkurborg         old                     310.888889   
Sólvallagata     Reykjavíkurborg         new                     317.117647   
Arkarvogur       Reykjavíkurborg         old                     321.333333   
Dalsmúli         Reykjavíkurborg         new                     332.000000   
Fellsmúli        Reykjavíkurborg         old                     332.500000   
Bæjarlind        Kópavogsbær             old                     332.500000   
Vitastígur       Reykjavíkurborg         old                     339.500000   
Malarskarð       Hafnarfjarðarkaupstaður new                     346.000000   
Hringhamar       Hafnarfjarðarkaupstaður old                     370.615385   
Grjótasel        Reykjavíkurborg         old                     372.500000   
Bygggarðar       Seltjarnarnesbær        new                     377.200000   
Áshamar          Hafnarfjarðarkaupstaður new                     384.132979   
Lautargata       Garðabær                new                     397.000000   
Borgartún        Reykjavíkurborg         old                     398.400000   
Naustavör        Kópavogsbær             new                     419.000000   
Bergsskarð       Hafnarfjarðarkaupstaður new                     430.000000   
Kolagata         Reykjavíkurborg         old                     434.000000   
Laugavegur       Reykjavíkurborg         new                     437.263158   
Grímsgata        Garðabær                new                     447.000000   
Brautarholt      Reykjavíkurborg         old                     447.333333   
Lambamýri        Garðabær                new                     464.142857   
Jöfursbás        Reykjavíkurborg         new                     473.000000   
Bankastræti      Reykjavíkurborg         old                     477.000000   
Ánanaust         Reykjavíkurborg         new                     482.600000   
Vinastræti       Garðabær                old                     487.500000   
Kinnargata       Garðabær                old                     489.333333   
Laugarásvegur    Reykjavíkurborg         old                     495.500000   
Áshamar          Hafnarfjarðarkaupstaður old                     511.157895   
Hlíðarfótur      Reykjavíkurborg         new                     557.500000   
Mýrargata        Reykjavíkurborg         new                     618.608696   

                                                                total  
street_name      city_name               construction_category         
Fálkahlíð        Reykjavíkurborg         new                       11  
Hávegur          Kópavogsbær             new                       11  
Kleppsmýrarvegur Reykjavíkurborg         new                       10  
Dugguvogur       Reykjavíkurborg         new                       14  
Álftröð          Kópavogsbær             new                       32  
Arkarvogur       Reykjavíkurborg         new                       11  
Apalskarð        Hafnarfjarðarkaupstaður old                        4  
                                         new                       23  
Bleikjukvísl     Reykjaví

In [14]:
df_by_h3 = df\
.assign(h3id = lambda r: r.apply(lambda x: h3.latlng_to_cell(x.latitude, x.longitude, 8), axis=1))\
.groupby('h3id')\
.agg(
    days_listed = ('days_listed', 'mean'),
    total = ('realestate_id', 'nunique')
)

In [22]:
colors = isb.plot.helpers.create_color_scaler(
    min_value=0, 
    max_value=360,
    color_low="#279638",
    color_high="#fd5a54"
)



fig = isb.Figure()

fig.add_densitymapbox(
    lat=df.longitude, 
    lon=df.latitude, 
    z=df.days_listed.apply(lambda x: 1),
    radius=7,
    opacity=0.5
)

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style="light",
        center=dict(
            lat=64.1024998,
            lon=-21.897828
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

In [21]:
colors = isb.plot.helpers.create_color_scaler(
    min_value=df_by_h3.total.min(), 
    max_value=df_by_h3.total.max(),
    color_low="#279638",
    color_high="#fd5a54"
)



fig = isb.Figure()

for i, row in df_by_h3.iterrows():

    boundary = h3.cell_to_boundary(i)

    fig.add_scattermapbox(
        lat=[b[1] for b in boundary],
        lon=[b[0] for b in boundary],
        mode='lines',
        line = dict(
            color = colors(row.total),
            width = 0
        ),
        fill='toself',
        showlegend=False
    )

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style="light",
        center=dict(
            lat=64.1024998,
            lon=-21.897828
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()